In [ ]:
import gradio as gr

import requests

import random

 

def gen_uuid():

    """

    Генерирует уникальный идентификатор.

   

    :return: Уникальный идентификатор

    """

    import uuid

    return uuid.uuid4()

 

sip_url = "http://127.0.0.1:8080"

examples=["Что ты знаешь о картографическом портале?"]

 

def print_like_dislike(chat_messages, x: gr.LikeData):

    """Выводит информацию о лайке или дизлайке.

   

    :param chat_messages: Список сообщений в чат-боте в объекте Gradio JSON

    :param x: Данные лайка или дизлайка

    """

    print(x.index, x.value, x.liked)

    if 'sip_interaction_id' in chat_messages[x.index[0]]['metadata']:

        print(chat_messages[x.index[0]]['metadata']['sip_interaction_id'], chat_messages[x.index[0]]['metadata']['session_id'])

        if x.liked:

            gr.Info("Спасибо за лайк! СИП будет и дальше стараться давать полезные ответы. Можете уточнить в форме обратной связи, чем именно вам понравился или помог ответ СИП")

        else:

            gr.Warning("Если не сложно, уточните в форме обратной связи, что именно не понравилось в ответе СИП")

    else:

        gr.Warning("Оценить лайком или дизлайком можно только ответы СИП на Ваши запросы")

 

def send_like_dislike(chat_messages, x: gr.LikeData):

    """Отправляет информацию о лайке или дизлайке.

   

    :param chat_messages: Список сообщений в чат-боте в объекте Gradio JSON

    :param x: Данные лайка или дизлайка

    """

    if 'sip_interaction_id' in chat_messages[x.index[0]]['metadata']:

        sip_respond = requests.post(f"{sip_url}/feedback/like", params={

            "sip_interaction_id": chat_messages[x.index[0]]['metadata']['sip_interaction_id'],

            "liked": x.liked})

        if sip_respond.status_code == 200:

            if x.liked:

                gr.Info("Спасибо за лайк! СИП будет и дальше стараться давать полезные ответы. Можете уточнить в форме обратной связи, чем именно вам понравился или помог ответ СИП")

            else:

                gr.Warning("Если не сложно, уточните в форме обратной связи, что именно не понравилось в ответе СИП")

        else:

            gr.Warning("К сожалению, оставить лайк сейчас нельзя :(")

            gr.Info(sip_respond.text)

    else:

        gr.Warning("Оценить лайком или дизлайком можно только ответы СИП на Ваши запросы")

 

def send_comment(feedback_comment, session_state):

    """Отправляет отзыв пользователя.

   

    :param feedback_comment: Текст отзыва пользователя

    :param session_state: Состояние сессии, содержащее ее параметры (такие как идентификатор сессии session_id)

    """

    sip_respond = requests.post(f"{sip_url}/feedback/comment", params={

        "session_id": session_state['session_id'],

        "comment_text": feedback_comment})

    if sip_respond.status_code == 200:

        gr.Info("Спасибо за комментарий работы СИП")

    else:

        gr.Warning("К сожалению, оставить отзыв сейчас нельзя :(")

        gr.Info(sip_respond.text)

 

def send_reference(feedback_reference_query, feedback_reference_answer, session_state):

    """Отправляет эталонный ответ.

   

    :param feedback_reference_query: Запрос

    :param feedback_reference_answer: Ответ

    :param session_state: Состояние сессии, содержащее ее параметры (такие как идентификатор сессии session_id)

    """

    sip_respond = requests.post(f"{sip_url}/feedback/reference", params={

        "session_id": session_state['session_id'],

        "reference_query": feedback_reference_query,

        "reference_answer": feedback_reference_answer}, stream=False)

    if sip_respond.status_code == 200:

        gr.Info("Спасибо за отправку эталонного ответа")

    else:

        gr.Warning("К сожалению, отправить эталонный ответ сейчас нельзя :(")

        gr.Info(sip_respond.text)

 

def send_grades(grade_help, grade_thinking, grade_speech, grade_help_example, grade_thinking_example, grade_speech_example, session_state):

    """Отправляет численные оценки пользователя по опыту взаимодействия с СИП.

   

    :param grade_help: Численная оценка того, насколько СИП помогает в поиске

    :param grade_thinking: Численная оценка того, как логично и разумно делает выводы СИП

    :param grade_speech: Численная оценка того, насколько связной является речь в ответах СИП

    :param grade_help_example: Текстовый пример, демонстрирующий насколько СИП помогает в поиске

    :param grade_thinking_example: Текстовый пример, демонстрирующий уровень логичности и разумности выводов СИП

    :param grade_speech_example: Текстовый пример, демонстрирующий уровень речи выводов СИП

    :param session_state: Состояние сессии, содержащее ее параметры (такие как идентификатор сессии session_id)

    """

    sip_respond = requests.post(f"{sip_url}/feedback/grades", params={

        "session_id": session_state['session_id'],

        "grade_help": grade_help,

        "grade_thinking": grade_thinking,

        "grade_speech": grade_speech,

        "grade_help_example": grade_help_example,

        "grade_thinking_example": grade_thinking_example,

        "grade_speech_example": grade_speech_example,

        })

    if sip_respond.status_code == 200:

        gr.Info("Спасибо за оценки работы СИП")

    else:

        gr.Warning("К сожалению оценить СИП сейчас нельзя :(")

        gr.Info(sip_respond.text)

 

def bot_init(history, chat_messages, uuid_store):

    """

    Инициализация чата с СИП.

 

    :param history: История чата

    :param chat_messages: Список сообщений в чат

    :param uuid_store: Объект, хранящий уникальный идентификатор сессии

    :return: Обновленная история диалога, обновленный список сообщений и объект, хранящий уникальный идентификатор сессии

    """

    uuid_store["session_id"] = str(gen_uuid())

    chat_messages += [{"role":"assistant", "content": """

                       **СИП** — это **помощник**, обученный на базе **ЛНД Общества**, но не безоговорочный источник истины. [Инструкция](https://knipi-portal/Pages/static/sip/guide.aspx). Все ответы не являются официальными мнениями **Компании** и **Общества**. Используя СИП, помните об условиях [пользовательского соглашения](https://knipi-portal/Pages/static/sip/terms.aspx). Выберите **тип поиска** и напишите свой вопрос **как человеку**. Например:*"Какую обувь можно носить в пятницу в офисе?"* """,

                          "metadata": {"session_id": uuid_store["session_id"]}}]

 

    history = chat_messages

    return history, chat_messages, uuid_store

 

def add_message(history, chat_messages, message, session_state):

    """

    Добавляет новое сообщение в чат.

 

    :param history: История чата.

    :param chat_messages: Список сообщений в чате.

    :param message: Новое сообщение, которое нужно добавить в чат.

    :param session_state: Состояние сессии, содержащее её параметры (например, идентификатор сессии).

 

    :return: Обновлённый список сообщений.

    """

    if len(message) > 16000:

        error_message = "Слишком длинный запрос"

        chat_messages += [{"role":"assistant", "content": error_message, "metadata": {"session_id": session_state['session_id']}}]

        gr.Warning(error_message)

    else:

        chat_messages += [{"role":"user", "content":message, "metadata": {"session_id": session_state['session_id']}}]

    history = chat_messages

    return history, chat_messages, gr.Textbox()

 

def bot(history, chat_messages, response_type, session_state):

    """

    Обрабатывает запрос пользователя.

 

    :param history: История чата.

    :param chat_messages: Список сообщений в чате.

    :param response_type: Тип ответа (поиск похожих ЛНД, поиск по ЛНД, чат с GPT-моделью).

    :param session_state: Состояние сессии, содержащее её параметры (например, идентификатор сессии).

 

    :return: Обновлённый список сообщений в чате.

    """

 

    if history[-1]["role"] != "user":

        return history, chat_messages

    # Обработка запроса в зависимости от выбранного типа поиска

    if response_type == "Чат со знанием ЛНД":

        # Час в режиме свободного общения с контекстом ЛНД

        sip_respond = requests.get(f"{sip_url}/sip", params={"query": history[-1]['content'], "mode": "lnd_chat", "session_id": session_state['session_id']})

    elif response_type == "Поиск похожих ЛНД":

        # Поиск похожих ЛНД

        sip_respond = requests.get(f"{sip_url}/sip", params={"query": history[-1]['content'], "mode": "lnd", "session_id": session_state['session_id']})

    elif response_type == "Поиск ЛНД по номеру":

        # Поиск ЛНД по номеру

        sip_respond = requests.get(f"{sip_url}/sip", params={"query": history[-1]['content'], "mode": "lnd_id", "session_id": session_state['session_id']})

    else:

        # Локальный чат-GPT

        sip_respond = requests.get(f"{sip_url}/sip", params={"query": history[-1]['content'], "mode": "llm_chat", "session_id": session_state['session_id']})

    sip_respond = sip_respond.json()

 

    # Обработка ответа СИП

    for message in sip_respond["messages"]:

        chat_messages += [{

            "role": "assistant",

            "content": f"""{message}""", #добавить выделение жирным

            "metadata": {

                "sip_interaction_id": sip_respond['sip_interaction_id'],

                "session_id": session_state['session_id']

            }

        }]

 

    # Добавление дополнительных сообщений из полученных объектов, если они доступны

    if "data_objects" in sip_respond:

        for data_object in sip_respond["data_objects"]:

            if data_object['metadata']['_collection_name'] == "lnd_chunk":

                # Добавление фрагмента ЛНД

                legal_entity_text = data_object['metadata']['legal_entity'] if data_object['metadata']['legal_entity'] != "Компания" else "Компании"

                lnd_title = f'''{data_object['metadata']['lnd_type']} {legal_entity_text} № {data_object['metadata']['id']} «{data_object['metadata']['lnd_name']}»'''

#                chat_messages += [{"role": "assistant", "content": f"""<p style="font-size: 10px; line-height: 0.5;">[{lnd_title}]({data_object['metadata']['lnd_link']}).</p>"""}]

                chunk_content = f"""**Фрагмент (семантическая близость: {data_object['metadata']['score']})** \n <span style="font-size: 20px; line-height: 1;">[{lnd_title}]({data_object['metadata']['lnd_link']})</span> \n {data_object['text'].strip()}"""

                chunk_title = f"""Фрагмент из ЛНД - {lnd_title}"""

                chat_messages += [{"role": "assistant", "content": chunk_content, "metadata": {"title": chunk_title}}]

            if data_object['metadata']['_collection_name'] in ["lnd", "lnd_description"]:

                # Добавление описания ЛНД

                legal_entity_text = data_object['metadata']['legal_entity'] if data_object['metadata']['legal_entity'] != "Компания" else "Компании"

                lnd_title = f'''{data_object['metadata']['lnd_type']} {legal_entity_text} № {data_object['metadata']['id']} «{data_object['metadata']['lnd_name']}»'''

                chat_messages += [{"role": "assistant", "content": f"""[{lnd_title}]({data_object['metadata']['lnd_link']})"""}]

                #chat_messages += [{"role": "assistant", "content": f"""<span style="font-size: 10px; line-height: 0.5;">{data_object['text'].strip()}.</span>"""}]

                chat_messages += [{"role": "assistant", "content": data_object['text'], "metadata": {"title":  "Описание"}}]

            if data_object['metadata']['_collection_name'] == "lnd_name":

                # Добавление названия ЛНД

                legal_entity_text = data_object['metadata']['legal_entity'] if data_object['metadata']['legal_entity'] != "Компания" else "Компании"

                lnd_title = f'''{data_object['metadata']['lnd_type']} {legal_entity_text} № {data_object['metadata']['id']} «{data_object['text']}» ({data_object['metadata']['lnd_link']})'''

                chat_messages += [{"role": "assistant", "content": lnd_title}]

                chat_messages += [{"role": "assistant", "content": data_object['metadata']['lnd_description'], "metadata": {"title":  "Описание"}}]

            if data_object['metadata']['_collection_name'] == "lnd_id":

                # Добавление описания ЛНД

                legal_entity_text = data_object['metadata']['legal_entity'] if data_object['metadata']['legal_entity'] != "Компания" else "Компании"

                lnd_title = f'''{data_object['metadata']['lnd_type']} {legal_entity_text} № {data_object['metadata']['id']} «{data_object['metadata']['lnd_name']}»'''

                chat_messages += [{"role": "assistant", "content": f"""[{lnd_title}]({data_object['metadata']['lnd_link']})"""}]

                #chat_messages += [{"role": "assistant", "content": f"""<span style="font-size: 10px; line-height: 0.5;">{data_object['text'].strip()}.</span>"""}]

                chat_messages += [{"role": "assistant", "content": data_object['metadata']['lnd_description'], "metadata": {"title":  "Описание"}}]

 

    # Обновление истории диалога

    history = chat_messages

   

    return history, chat_messages

 

# Стиль интерфейса

css = """

    footer { display: none !important; } .app.svelte-wpkpf6.svelte-wpkpf6 {padding: 0;} .tabitem { border: 0; }

"""

#body_text_color = '*neutral_950', body_text_color_subdued = '*neutral_950' body_text_weight = 400,

theme = gr.themes.Default(secondary_hue='yellow', neutral_hue="stone").set(body_text_color = '*neutral_950',

                                                                           prose_text_weight = 500,

                                                                           prose_header_text_weight = 600,

                                                                           border_color_primary='*neutral_400' #рамочки вокруг объектов

                                                                           )

 

with gr.Blocks(theme=theme, fill_height=True, fill_width=True, css=css).queue(default_concurrency_limit=None) as demo:

    """

    gr.Blocks - Объект для создания интерактивных блоков

    

    :param theme: Тема оформления блока

    :param fill_height: Значение для атрибута `height` блока

    :param fill_width: Значение для атрибута `width` блока

    :param css: CSS-стили для блока

 

    Метод queue(default_concurrency_limit=None) позволяет выполнять асинхронные запросы из разных открытых экземпляров интерфейса, не создавая очередь.

    """

 

    # Вкладка с основным функционалом

    with gr.Tab("Система интеллектуального поиска () v1.0.1"):

        #gr.HTML(value="Система интеллектуального поиска () v0.1.0") # Зазоловик (сейчас скрыт!)

        # Аккордион со сужебными полями для мониторинга объектов сессии (сейчас скрыт!)

        with gr.Accordion("@@@ Тестовые поля", open=False, visible=False):

            session_state = gr.JSON({})

            chat_messages = gr.JSON(value = [], visible=True)

        

        # Чат для отображения сообщений полязивателя и ответов СИП

        chatbot = gr.Chatbot(

            label="Диалог с СИП",

            elem_id="chatbot",

            type='messages',

            bubble_full_width=True,

            show_copy_button=True,

            scale=5,

            height= "550px"

           

        )

       

        # Выбор режима поиска

        response_type = gr.Radio(["Чат со знанием ЛНД", "Поиск похожих ЛНД", "Поиск ЛНД по номеру", "Чат с GPT-моделью"], value="Чат со знанием ЛНД", label="Тип поиска")

 

        # Элементы для ввода запросов пользователя

        with gr.Row():

            chat_input = gr.Textbox(container=False, lines=1, placeholder=f"Ваш запрос (например: {random.choice(examples)})", show_label=True, show_copy_button=True, scale=8)

            submit = gr.Button(value = "Отправить", scale=1)

            # clear = gr.ClearButton([chat_input], value = "Очистить", scale=1) убираем кнопку очистить

            # clear.click(lambda: "", None, chat_messages)

 

        # Нажатие Enter после ввода запроса в текстовое поле (== Клик на кнопку "Отправить")

        chat_msg = chat_input.submit(add_message, [chatbot, chat_messages, chat_input, session_state], [chatbot, chat_messages, chat_input])

        bot_msg =chat_msg.then(lambda: "", None, [chat_input]) # Очищаем Поле ввода

        bot_msg = bot_msg.then(bot, [chatbot, chat_messages, response_type, session_state], [chatbot, chat_messages], api_name="bot_response")

 

        # Клик на кнопку "Отправить"

        chat_msg_click = submit.click(add_message, [chatbot, chat_messages, chat_input, session_state], [chatbot, chat_messages, chat_input])

        bot_msg_click = chat_msg_click.then(lambda: "", None, [chat_input]) #gr.Textbox()

        bot_msg_click = bot_msg_click.then(bot, [chatbot, chat_messages, response_type, session_state], [chatbot, chat_messages], api_name="bot_response")

 

        # Активация возможности ставить лайк/дизлайк

        chatbot.like(send_like_dislike, chat_messages, None)

 

        # Виджет с примерам (сейчас скрыт!)

        gr.Examples(examples=examples, inputs=[chat_input], visible=False)

 

    # Вкладка с обратной связью

    with gr.Tab("Обратная связь"):#gr.Accordion("Обратная связь", open=False):

 

        # Виджеты отправки комментария/отзыва по работе СИП

        feedback_comment = gr.TextArea(label="Оставьте свой отзыв о работе СИП (Напишите комментарий, отзыв или предложение)")

        feedback_comment_btn = gr.Button(value="Отправить отзыв")

        feedback_comment_btn.click(fn=send_comment, inputs=[feedback_comment, session_state])

 

        # Виджеты отправки оценок работы СИП по численным критериям

        with gr.Column():

            gr.HTML(value="Можете прикрепить пример, демонстрирующий вашу оценку, в текстовое окно под шкалой соответвующей оценки")

            with gr.Row():

                grade_help = gr.Slider(0, 5, step=1, info="Оцените насколько СИП помог в поиске нужной информации", label='Помощь')

                grade_thinking = gr.Slider(0, 5, step=1, info="Оцените точность и разумность выводов на основе приведенной информации", label='Точность')

                grade_speech = gr.Slider(0, 5, step=1, info="Оцените качество ответов", label='Речь')

            with gr.Row():

                grade_help_example = gr.TextArea(container=False, lines=2, placeholder="Пример текста, который помог в поиске решения Вашей проблемы")

                grade_thinking_example = gr.TextArea(container=False, lines=2, placeholder="Пример текста, отражающий правильное или ошибочное рассуждение")

                grade_speech_example = gr.TextArea(container=False, lines=2, placeholder="Пример текста, в котором проявлен указанный уровень речи")

        feedback_grades_btn = gr.Button(value="Отправить оценки")

        feedback_grades_btn.click(fn=send_grades, inputs=[grade_help, grade_thinking, grade_speech,

                                                          grade_help_example, grade_thinking_example, grade_speech_example, session_state])

       

        # Виджеты отправки текста запроса и верного по мнению пользователя ответа на запрос

        gr.HTML(value="Можете поделиться эталонным ответом на Ваш запрос")

        with gr.Row():

            feedback_reference_query = gr.TextArea(container=False, placeholder="Введите текст запроса", lines=3, label='Запрос')

            feedback_reference_answer = gr.TextArea(container=False, placeholder="Введите эталонный ответ", lines=3, label='Ответ')

            #user_certainty = gr.Slider(0, 5, step=1, info="Оцените вашу уверенность что ваша формулировка однозначно отвечает на поставаленный запрос", label='Уверенность')

        feedback_reference_btn = gr.Button(value="Отправить эталонный ответ")

        feedback_reference_btn.click(fn=send_reference, inputs=[feedback_reference_query, feedback_reference_answer, session_state])

 

    """

    Загрузка интерфейса. Позволяет при каждом новом открытие окна с СИП UI в каком-либо браузере исполнять исполнять функцию (bot_init), и таким образом генерировать свой новый идентификатор сессии.

 

    :param fn: Функция, которая будет вызвана при загрузке интерфейса.

   :param inputs: Список входных данных для функции.

    :param outputs: Список выходных данных функции.

    """

    demo.load(bot_init, [chatbot, chat_messages, session_state], [chatbot, chat_messages, session_state])

 

if __name__ == "__main__":

    # Запуск интерфейса

    demo.launch(server_name='0.0.0.0', server_port=7861)

In [ ]:
import gradio as gr
import requests
import random
import pyaudio
import wave
import threading
import time
import os

# Конфигурация записи (из первого интерфейса)
RECORD_SECONDS = 15
CHUNK = 1024
FORMAT = pyaudio.paInt32
CHANNELS = 1
RATE = 44100
MICROSERVICE_URL = "http://localhost:8000/transcribe/"

# Глобальные переменные для записи
is_recording = False
frames = []
stream = None
p = None

def gen_uuid():
    """Генерирует уникальный идентификатор."""
    import uuid
    return uuid.uuid4()

sip_url = "http://127.0.0.1:8080"
examples=["Что ты знаешь о картографическом портале?"]

# Функции для записи аудио (из первого интерфейса)
def wav_to_binary(wav_file_path):
    """Конвертирует WAV в бинарный файл"""
    binary_file_path = os.path.splitext(wav_file_path)[0] + '.bin'
    try:
        with open(wav_file_path, 'rb') as wav_file, open(binary_file_path, 'wb') as binary_file:
            binary_file.write(wav_file.read())
        return binary_file_path
    except Exception as e:
        print(f"Ошибка конвертации: {e}")
        return None

def send_to_microservice(binary_file_path):
    """Отправляет бинарный файл на микросервис транскрибации и возвращает результат"""
    try:
        with open(binary_file_path, 'rb') as f:
            files = {'file': (os.path.basename(binary_file_path), f, 'application/octet-stream')}
            response = requests.post(MICROSERVICE_URL, files=files)
        
        if response.status_code == 200:
            result = response.json()
            return result.get('transcription', 'Текст не найден в ответе')
        else:
            return f"Ошибка сервера: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Ошибка отправки: {str(e)}"

def record_audio(output_filename, chat_input):
    global is_recording, frames, stream, p
    
    if not is_recording:
        # Начало записи
        is_recording = True
        frames = []
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT, channels=CHANNELS, 
                       rate=RATE, input=True,
                       frames_per_buffer=CHUNK)
        
        def recording_thread():
            global is_recording, frames
            for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
                if not is_recording:
                    break
                data = stream.read(CHUNK, exception_on_overflow=False)
                frames.append(data)
            is_recording = False
        
        threading.Thread(target=recording_thread).start()
        return "Запись...", chat_input
    else:
        # Остановка записи
        is_recording = False
        time.sleep(0.1)
        
        if stream:
            stream.stop_stream()
            stream.close()
        if p:
            p.terminate()
        
        result_text = chat_input  # Используем текущий текст из поля ввода по умолчанию
        
        if frames:
            # Сохраняем WAV
            with wave.open(output_filename, 'wb') as wf:
                wf.setnchannels(CHANNELS)
                wf.setsampwidth(p.get_sample_size(FORMAT))
                wf.setframerate(RATE)
                wf.writeframes(b''.join(frames))
            
            # Конвертируем в бинарный
            binary_path = wav_to_binary(output_filename)
            if binary_path:
                # Отправляем на микросервис и получаем результат
                result_text = send_to_microservice(binary_path)
            
            duration = len(frames)/RATE
            status = f"Запись сохранена ({duration:.1f} сек)"
            return status, result_text
        
        return "Запись отменена", chat_input

def print_like_dislike(chat_messages, x: gr.LikeData):

    """Выводит информацию о лайке или дизлайке.

   

    :param chat_messages: Список сообщений в чат-боте в объекте Gradio JSON

    :param x: Данные лайка или дизлайка

    """

    print(x.index, x.value, x.liked)

    if 'sip_interaction_id' in chat_messages[x.index[0]]['metadata']:

        print(chat_messages[x.index[0]]['metadata']['sip_interaction_id'], chat_messages[x.index[0]]['metadata']['session_id'])

        if x.liked:

            gr.Info("Спасибо за лайк! СИП будет и дальше стараться давать полезные ответы. Можете уточнить в форме обратной связи, чем именно вам понравился или помог ответ СИП")

        else:

            gr.Warning("Если не сложно, уточните в форме обратной связи, что именно не понравилось в ответе СИП")

    else:

        gr.Warning("Оценить лайком или дизлайком можно только ответы СИП на Ваши запросы")

 

def send_like_dislike(chat_messages, x: gr.LikeData):

    """Отправляет информацию о лайке или дизлайке.

   

    :param chat_messages: Список сообщений в чат-боте в объекте Gradio JSON

    :param x: Данные лайка или дизлайка

    """

    if 'sip_interaction_id' in chat_messages[x.index[0]]['metadata']:

        sip_respond = requests.post(f"{sip_url}/feedback/like", params={

            "sip_interaction_id": chat_messages[x.index[0]]['metadata']['sip_interaction_id'],

            "liked": x.liked})

        if sip_respond.status_code == 200:

            if x.liked:

                gr.Info("Спасибо за лайк! СИП будет и дальше стараться давать полезные ответы. Можете уточнить в форме обратной связи, чем именно вам понравился или помог ответ СИП")

            else:

                gr.Warning("Если не сложно, уточните в форме обратной связи, что именно не понравилось в ответе СИП")

        else:

            gr.Warning("К сожалению, оставить лайк сейчас нельзя :(")

            gr.Info(sip_respond.text)

    else:

        gr.Warning("Оценить лайком или дизлайком можно только ответы СИП на Ваши запросы")

 

def send_comment(feedback_comment, session_state):

    """Отправляет отзыв пользователя.

   

    :param feedback_comment: Текст отзыва пользователя

    :param session_state: Состояние сессии, содержащее ее параметры (такие как идентификатор сессии session_id)

    """

    sip_respond = requests.post(f"{sip_url}/feedback/comment", params={

        "session_id": session_state['session_id'],

        "comment_text": feedback_comment})

    if sip_respond.status_code == 200:

        gr.Info("Спасибо за комментарий работы СИП")

    else:

        gr.Warning("К сожалению, оставить отзыв сейчас нельзя :(")

        gr.Info(sip_respond.text)

 

def send_reference(feedback_reference_query, feedback_reference_answer, session_state):

    """Отправляет эталонный ответ.

   

    :param feedback_reference_query: Запрос

    :param feedback_reference_answer: Ответ

    :param session_state: Состояние сессии, содержащее ее параметры (такие как идентификатор сессии session_id)

    """

    sip_respond = requests.post(f"{sip_url}/feedback/reference", params={

        "session_id": session_state['session_id'],

        "reference_query": feedback_reference_query,

        "reference_answer": feedback_reference_answer}, stream=False)

    if sip_respond.status_code == 200:

        gr.Info("Спасибо за отправку эталонного ответа")

    else:

        gr.Warning("К сожалению, отправить эталонный ответ сейчас нельзя :(")

        gr.Info(sip_respond.text)

 

def send_grades(grade_help, grade_thinking, grade_speech, grade_help_example, grade_thinking_example, grade_speech_example, session_state):

    """Отправляет численные оценки пользователя по опыту взаимодействия с СИП.

   

    :param grade_help: Численная оценка того, насколько СИП помогает в поиске

    :param grade_thinking: Численная оценка того, как логично и разумно делает выводы СИП

    :param grade_speech: Численная оценка того, насколько связной является речь в ответах СИП

    :param grade_help_example: Текстовый пример, демонстрирующий насколько СИП помогает в поиске

    :param grade_thinking_example: Текстовый пример, демонстрирующий уровень логичности и разумности выводов СИП

    :param grade_speech_example: Текстовый пример, демонстрирующий уровень речи выводов СИП

    :param session_state: Состояние сессии, содержащее ее параметры (такие как идентификатор сессии session_id)

    """

    sip_respond = requests.post(f"{sip_url}/feedback/grades", params={

        "session_id": session_state['session_id'],

        "grade_help": grade_help,

        "grade_thinking": grade_thinking,

        "grade_speech": grade_speech,

        "grade_help_example": grade_help_example,

        "grade_thinking_example": grade_thinking_example,

        "grade_speech_example": grade_speech_example,

        })

    if sip_respond.status_code == 200:

        gr.Info("Спасибо за оценки работы СИП")

    else:

        gr.Warning("К сожалению оценить СИП сейчас нельзя :(")

        gr.Info(sip_respond.text)

 

def bot_init(history, chat_messages, uuid_store):

    """

    Инициализация чата с СИП.

 

    :param history: История чата

    :param chat_messages: Список сообщений в чат

    :param uuid_store: Объект, хранящий уникальный идентификатор сессии

    :return: Обновленная история диалога, обновленный список сообщений и объект, хранящий уникальный идентификатор сессии

    """

    uuid_store["session_id"] = str(gen_uuid())

    chat_messages += [{"role":"assistant", "content": """

                       **СИП** — это **помощник**, обученный на базе **ЛНД Общества**, но не безоговорочный источник истины. [Инструкция](https://knipi-portal/Pages/static/sip/guide.aspx). Все ответы не являются официальными мнениями **Компании** и **Общества**. Используя СИП, помните об условиях [пользовательского соглашения](https://knipi-portal/Pages/static/sip/terms.aspx). Выберите **тип поиска** и напишите свой вопрос **как человеку**. Например:*"Какую обувь можно носить в пятницу в офисе?"* """,

                          "metadata": {"session_id": uuid_store["session_id"]}}]

 

    history = chat_messages

    return history, chat_messages, uuid_store

 

def add_message(history, chat_messages, message, session_state):

    """

    Добавляет новое сообщение в чат.

 

    :param history: История чата.

    :param chat_messages: Список сообщений в чате.

    :param message: Новое сообщение, которое нужно добавить в чат.

    :param session_state: Состояние сессии, содержащее её параметры (например, идентификатор сессии).

 

    :return: Обновлённый список сообщений.

    """

    if len(message) > 16000:

        error_message = "Слишком длинный запрос"

        chat_messages += [{"role":"assistant", "content": error_message, "metadata": {"session_id": session_state['session_id']}}]

        gr.Warning(error_message)

    else:

        chat_messages += [{"role":"user", "content":message, "metadata": {"session_id": session_state['session_id']}}]

    history = chat_messages

    return history, chat_messages, gr.Textbox()

 

def bot(history, chat_messages, response_type, session_state):

    """

    Обрабатывает запрос пользователя.

 

    :param history: История чата.

    :param chat_messages: Список сообщений в чате.

    :param response_type: Тип ответа (поиск похожих ЛНД, поиск по ЛНД, чат с GPT-моделью).

    :param session_state: Состояние сессии, содержащее её параметры (например, идентификатор сессии).

 

    :return: Обновлённый список сообщений в чате.

    """

 

    if history[-1]["role"] != "user":

        return history, chat_messages

    # Обработка запроса в зависимости от выбранного типа поиска

    if response_type == "Чат со знанием ЛНД":

        # Час в режиме свободного общения с контекстом ЛНД

        sip_respond = requests.get(f"{sip_url}/sip", params={"query": history[-1]['content'], "mode": "lnd_chat", "session_id": session_state['session_id']})

    elif response_type == "Поиск похожих ЛНД":

        # Поиск похожих ЛНД

        sip_respond = requests.get(f"{sip_url}/sip", params={"query": history[-1]['content'], "mode": "lnd", "session_id": session_state['session_id']})

    elif response_type == "Поиск ЛНД по номеру":

        # Поиск ЛНД по номеру

        sip_respond = requests.get(f"{sip_url}/sip", params={"query": history[-1]['content'], "mode": "lnd_id", "session_id": session_state['session_id']})

    else:

        # Локальный чат-GPT

        sip_respond = requests.get(f"{sip_url}/sip", params={"query": history[-1]['content'], "mode": "llm_chat", "session_id": session_state['session_id']})

    sip_respond = sip_respond.json()

 

    # Обработка ответа СИП

    for message in sip_respond["messages"]:

        chat_messages += [{

            "role": "assistant",

            "content": f"""{message}""", #добавить выделение жирным

            "metadata": {

                "sip_interaction_id": sip_respond['sip_interaction_id'],

                "session_id": session_state['session_id']

            }

        }]

 

    # Добавление дополнительных сообщений из полученных объектов, если они доступны

    if "data_objects" in sip_respond:

        for data_object in sip_respond["data_objects"]:

            if data_object['metadata']['_collection_name'] == "lnd_chunk":

                # Добавление фрагмента ЛНД

                legal_entity_text = data_object['metadata']['legal_entity'] if data_object['metadata']['legal_entity'] != "Компания" else "Компании"

                lnd_title = f'''{data_object['metadata']['lnd_type']} {legal_entity_text} № {data_object['metadata']['id']} «{data_object['metadata']['lnd_name']}»'''

#                chat_messages += [{"role": "assistant", "content": f"""<p style="font-size: 10px; line-height: 0.5;">[{lnd_title}]({data_object['metadata']['lnd_link']}).</p>"""}]

                chunk_content = f"""**Фрагмент (семантическая близость: {data_object['metadata']['score']})** \n <span style="font-size: 20px; line-height: 1;">[{lnd_title}]({data_object['metadata']['lnd_link']})</span> \n {data_object['text'].strip()}"""

                chunk_title = f"""Фрагмент из ЛНД - {lnd_title}"""

                chat_messages += [{"role": "assistant", "content": chunk_content, "metadata": {"title": chunk_title}}]

            if data_object['metadata']['_collection_name'] in ["lnd", "lnd_description"]:

                # Добавление описания ЛНД

                legal_entity_text = data_object['metadata']['legal_entity'] if data_object['metadata']['legal_entity'] != "Компания" else "Компании"

                lnd_title = f'''{data_object['metadata']['lnd_type']} {legal_entity_text} № {data_object['metadata']['id']} «{data_object['metadata']['lnd_name']}»'''

                chat_messages += [{"role": "assistant", "content": f"""[{lnd_title}]({data_object['metadata']['lnd_link']})"""}]

                #chat_messages += [{"role": "assistant", "content": f"""<span style="font-size: 10px; line-height: 0.5;">{data_object['text'].strip()}.</span>"""}]

                chat_messages += [{"role": "assistant", "content": data_object['text'], "metadata": {"title":  "Описание"}}]

            if data_object['metadata']['_collection_name'] == "lnd_name":

                # Добавление названия ЛНД

                legal_entity_text = data_object['metadata']['legal_entity'] if data_object['metadata']['legal_entity'] != "Компания" else "Компании"

                lnd_title = f'''{data_object['metadata']['lnd_type']} {legal_entity_text} № {data_object['metadata']['id']} «{data_object['text']}» ({data_object['metadata']['lnd_link']})'''

                chat_messages += [{"role": "assistant", "content": lnd_title}]

                chat_messages += [{"role": "assistant", "content": data_object['metadata']['lnd_description'], "metadata": {"title":  "Описание"}}]

            if data_object['metadata']['_collection_name'] == "lnd_id":

                # Добавление описания ЛНД

                legal_entity_text = data_object['metadata']['legal_entity'] if data_object['metadata']['legal_entity'] != "Компания" else "Компании"

                lnd_title = f'''{data_object['metadata']['lnd_type']} {legal_entity_text} № {data_object['metadata']['id']} «{data_object['metadata']['lnd_name']}»'''

                chat_messages += [{"role": "assistant", "content": f"""[{lnd_title}]({data_object['metadata']['lnd_link']})"""}]

                #chat_messages += [{"role": "assistant", "content": f"""<span style="font-size: 10px; line-height: 0.5;">{data_object['text'].strip()}.</span>"""}]

                chat_messages += [{"role": "assistant", "content": data_object['metadata']['lnd_description'], "metadata": {"title":  "Описание"}}]

 

    # Обновление истории диалога

    history = chat_messages

   

    return history, chat_messages

 


css = """
    footer { display: none !important; } 
    .app.svelte-wpkpf6.svelte-wpkpf6 {padding: 0;} 
    .tabitem { border: 0; }
    .microphone-btn {
        min-width: 40px !important;
        width: 40px !important;
        height: 40px !important;
        border-radius: 100% !important;
        padding: 0 !important;
        display: flex;
        align-items: center;
        justify-content: center;
        aspect-ratio: 1/1 !important;
    }
    .microphone-btn svg {
        width: 20px !important;
        height: 20px !important;
    }
"""

theme = gr.themes.Default(secondary_hue='yellow', neutral_hue="stone").set(
    body_text_color='*neutral_950',
    prose_text_weight=500,
    prose_header_text_weight=600,
    border_color_primary='*neutral_400'
)

with gr.Blocks(theme=theme, fill_height=True, fill_width=True, css=css).queue(default_concurrency_limit=None) as demo:
    with gr.Tab("Система интеллектуального поиска () v1.0.1"):
        with gr.Accordion("@@@ Тестовые поля", open=False, visible=False):
            session_state = gr.JSON({})
            chat_messages = gr.JSON(value=[], visible=True)
        
        chatbot = gr.Chatbot(
            label="Диалог с СИП",
            elem_id="chatbot",
            type='messages',
            bubble_full_width=True,
            show_copy_button=True,
            scale=5,
            height="550px"
        )
       
        response_type = gr.Radio(
            ["Чат со знанием ЛНД", "Поиск похожих ЛНД", "Поиск ЛНД по номеру", "Чат с GPT-моделью"], 
            value="Чат со знанием ЛНД", 
            label="Тип поиска"
        )

        with gr.Row():
            chat_input = gr.Textbox(
                container=False, 
                lines=1, 
                placeholder=f"Ваш запрос (например: {random.choice(examples)})", 
                show_label=True, 
                show_copy_button=True, 
                scale=8
            )
           
            record_btn = gr.Button(
                value="🎤", 
                scale=1,
                elem_classes="microphone-btn"
            )
            submit = gr.Button(value="Отправить", scale=1)
            
        # Скрытый input для автоматического добавления .wav
        wav_input = gr.Textbox(value="recording.wav", visible=False)
        
        # Обработка клика на кнопку микрофона
        record_btn.click(
            fn=record_audio,
            inputs=[wav_input, chat_input],
            outputs=[gr.Textbox(visible=False), chat_input]
        )
        
        # Остальные обработчики из второго интерфейса
        chat_msg = chat_input.submit(add_message, [chatbot, chat_messages, chat_input, session_state], [chatbot, chat_messages, chat_input])
        bot_msg = chat_msg.then(lambda: "", None, [chat_input])
        bot_msg = bot_msg.then(bot, [chatbot, chat_messages, response_type, session_state], [chatbot, chat_messages], api_name="bot_response")

        chat_msg_click = submit.click(add_message, [chatbot, chat_messages, chat_input, session_state], [chatbot, chat_messages, chat_input])
        bot_msg_click = chat_msg_click.then(lambda: "", None, [chat_input])
        bot_msg_click = bot_msg_click.then(bot, [chatbot, chat_messages, response_type, session_state], [chatbot, chat_messages], api_name="bot_response")

        chatbot.like(send_like_dislike, chat_messages, None)
        gr.Examples(examples=examples, inputs=[chat_input], visible=False)

    # Вкладка с обратной связью (без изменений)
    with gr.Tab("Обратная связь"):
        feedback_comment = gr.TextArea(label="Оставьте свой отзыв о работе СИП (Напишите комментарий, отзыв или предложение)")
        feedback_comment_btn = gr.Button(value="Отправить отзыв")
        feedback_comment_btn.click(fn=send_comment, inputs=[feedback_comment, session_state])

        with gr.Column():
            gr.HTML(value="Можете прикрепить пример, демонстрирующий вашу оценку, в текстовое окно под шкалой соответвующей оценки")
            with gr.Row():
                grade_help = gr.Slider(0, 5, step=1, info="Оцените насколько СИП помог в поиске нужной информации", label='Помощь')
                grade_thinking = gr.Slider(0, 5, step=1, info="Оцените точность и разумность выводов на основе приведенной информации", label='Точность')
                grade_speech = gr.Slider(0, 5, step=1, info="Оцените качество ответов", label='Речь')
            with gr.Row():
                grade_help_example = gr.TextArea(container=False, lines=2, placeholder="Пример текста, который помог в поиске решения Вашей проблемы")
                grade_thinking_example = gr.TextArea(container=False, lines=2, placeholder="Пример текста, отражающий правильное или ошибочное рассуждение")
                grade_speech_example = gr.TextArea(container=False, lines=2, placeholder="Пример текста, в котором проявлен указанный уровень речи")
        feedback_grades_btn = gr.Button(value="Отправить оценки")
        feedback_grades_btn.click(fn=send_grades, inputs=[grade_help, grade_thinking, grade_speech,
                                                          grade_help_example, grade_thinking_example, grade_speech_example, session_state])
       
        gr.HTML(value="Можете поделиться эталонным ответом на Ваш запрос")
        with gr.Row():
            feedback_reference_query = gr.TextArea(container=False, placeholder="Введите текст запроса", lines=3, label='Запрос')
            feedback_reference_answer = gr.TextArea(container=False, placeholder="Введите эталонный ответ", lines=3, label='Ответ')
        feedback_reference_btn = gr.Button(value="Отправить эталонный ответ")
        feedback_reference_btn.click(fn=send_reference, inputs=[feedback_reference_query, feedback_reference_answer, session_state])

    demo.load(bot_init, [chatbot, chat_messages, session_state], [chatbot, chat_messages, session_state])

if __name__ == "__main__":
    demo.launch(server_name='0.0.0.0', server_port=7865)

C:\Users\vital\AppData\Local\Temp\ipykernel_19468\2678098250.py:568: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(
c:\Users\vital\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\base.py:196: UserWarning: show_label has no effect when container is False.
  warnings.warn("show_label has no effect when container is False.")


* Running on local URL:  http://0.0.0.0:7865

To create a public link, set `share=True` in `launch()`.
